<a href="https://colab.research.google.com/github/elangbijak4/NLP-AI-Algorithm/blob/main/Kombinasi_Norvig_dan_Damerau_Levenshtein_Distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
from collections import Counter

In [7]:
# Contoh korpus sederhana dalam bahasa Indonesia
korpus = """
anak api buku bawa cantik dan dari dia dengan gajah gunung hutan ibu ingin itu jalan
jauh karena kerja kembali laut makan minum naik panas rumah sangat suka sudah tidak
"""

# Fungsi untuk membaca korpus kata dari string teks bahasa Indonesia
def words(text):
    return re.findall(r'\w+', text.lower())

# Fungsi untuk membuat model probabilitas dari korpus kata
def train(words):
    model = Counter(words)
    return model

# Membuat model probabilitas dari korpus kata
NWORDS = train(words(korpus))

# Karakter alfabet untuk bahasa Indonesia
alphabet = 'abcdefghijklmnopqrstuvwxyz'

# Fungsi untuk menghasilkan semua kemungkinan kata yang dihasilkan dari satu operasi pengeditan
def edits1(word):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in alphabet]
    inserts = [L + c + R for L, R in splits for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

# Fungsi untuk menghasilkan semua kemungkinan kata yang dihasilkan dari dua operasi pengeditan
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

# Fungsi untuk mengembalikan kata yang ada dalam model
def known(words):
    return set(w for w in words if w in NWORDS)

# Fungsi untuk memperbaiki ejaan dengan menggunakan Norvig dan Damerau-Levenshtein Distance
def correct_with_norvig_and_damerau_levenshtein(word):
    candidates = (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
    return min(candidates, key=lambda x: damerau_levenshtein_distance(word, x))

# Fungsi untuk menghitung Damerau-Levenshtein Distance
def damerau_levenshtein_distance(s1, s2):
    m = len(s1)
    n = len(s2)

    # Inisialisasi matriks DP dengan ukuran (m+1) x (n+1)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Inisialisasi kolom pertama
    for i in range(1, m + 1):
        dp[i][0] = i

    # Inisialisasi baris pertama
    for j in range(1, n + 1):
        dp[0][j] = j

    # Mengisi matriks DP
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j],      # Hapus karakter dari s1
                                  dp[i][j - 1],      # Sisipkan karakter ke s1
                                  dp[i - 1][j - 1])  # Ganti karakter di s1

            # Menambahkan operasi transposisi
            if i > 1 and j > 1 and s1[i - 1] == s2[j - 2] and s1[i - 2] == s2[j - 1]:
                dp[i][j] = min(dp[i][j], dp[i - 2][j - 2] + 1)  # Transposisi

    return dp[m][n]

# Contoh penggunaan
word = "bukuui"
correction = correct_with_norvig_and_damerau_levenshtein(word)
print(f"Kata asli: {word}")
print(f"Koreksi: {correction}")

Kata asli: bukuui
Koreksi: buku
